In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from datetime import datetime
import csv
import functools
import warnings
warnings.filterwarnings('ignore') 
import os
from collections import defaultdict, deque
import paramiko
from functools import partial
import pickle
import redis
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('expand_frame_repr', True) # pycharm设置为False可以产生滑条，但vscode不能产生滑条所以会堆叠。因此vscode应该设置为True
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
# pd.set_option('display.max_rows', 100) # 显示的最大行数（避免只显示部分行数据）
pd.set_option('display.max_columns', 1000) # 显示的最大列数（避免列显示不全）
# pd.set_option("display.max_colwidth",1000) # 每一列最大的宽度（避免属性值或列名显示不全）
pd.set_option('display.width', 180) # 每一行的宽度（避免换行）
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (16, 10)
%config InlineBackend.figure_format = 'svg'
%matplotlib inline


In [2]:
from ctaUtils.product_info import *
from ctaUtils.helper import *
from ctaUtils.stats import *
from ctaUtils.RemoteQuery import *

In [7]:
rc = RemoteSrc()
l1_example = rc.get_raw_bars('v2108', '20210302', 'l1')
l2_example = rc.get_raw_bars('v2108', '20210302', 'l2')
l1_example.columns
l2_example.columns

Index(['date', 'code', 'server_time', 'local_time', 'preclose', 'open', 'high', 'low', 'last', 'upper_limit', 'lower_limit', 'volume', 'turnover', 'iopv', 'ask_price1',
       'ask_volume1', 'ask_price2', 'ask_volume2', 'ask_price3', 'ask_volume3', 'ask_price4', 'ask_volume4', 'ask_price5', 'ask_volume5', 'bid_price1', 'bid_volume1',
       'bid_price2', 'bid_volume2', 'bid_price3', 'bid_volume3', 'bid_price4', 'bid_volume4', 'bid_price5', 'bid_volume5', 'time'],
      dtype='object')

Index(['date', 'code', 'server_time', 'local_time', 'preclose', 'open', 'high', 'low', 'last', 'upper_limit', 'lower_limit', 'volume', 'turnover', 'iopv', 'ask_price1',
       'ask_volume1', 'ask_price2', 'ask_volume2', 'ask_price3', 'ask_volume3', 'ask_price4', 'ask_volume4', 'ask_price5', 'ask_volume5', 'bid_price1', 'bid_volume1',
       'bid_price2', 'bid_volume2', 'bid_price3', 'bid_volume3', 'bid_price4', 'bid_volume4', 'bid_price5', 'bid_volume5', 'time'],
      dtype='object')

In [8]:
l1_example.head()

,date,code,server_time,local_time,preclose,open,high,low,last,upper_limit,lower_limit,volume,turnover,iopv,ask_price1,ask_volume1,ask_price2,ask_volume2,ask_price3,ask_volume3,ask_price4,ask_volume4,ask_price5,ask_volume5,bid_price1,bid_volume1,bid_price2,bid_volume2,bid_price3,bid_volume3,bid_price4,bid_volume4,bid_price5,bid_volume5,time
0,20210302,v2108,2021-03-02 09:00:00.004,2021-03-02 08:59:55.529,8060,0,0,0,0,8610,7490,0,0,0,8105,1,8175,1,0,0,0,0,0,0,7880,11,7835,1,0,0,0,0,0,0,09:00:00.004000
1,20210302,v2108,2021-03-02 09:00:00.185,2021-03-02 08:59:55.733,8060,0,0,0,0,8610,7490,0,0,0,8100,2,8175,1,0,0,0,0,0,0,7880,11,7835,1,0,0,0,0,0,0,09:00:00.185000
2,20210302,v2108,2021-03-02 09:00:00.438,2021-03-02 08:59:55.978,8060,0,0,0,0,8610,7490,0,0,0,8295,1,0,0,0,0,0,0,0,0,7775,1,0,0,0,0,0,0,0,0,09:00:00.438000
3,20210302,v2108,2021-03-02 09:00:00.841,2021-03-02 08:59:56.210,8060,0,0,0,0,8610,7490,0,0,0,8095,3,8170,2,8295,1,0,0,0,0,7775,1,0,0,0,0,0,0,0,0,09:00:00.841000
4,20210302,v2108,2021-03-02 09:00:00.841,2021-03-02 08:59:56.465,8060,0,0,0,0,8610,7490,0,0,0,8095,3,8165,1,8295,1,0,0,0,0,7775,1,0,0,0,0,0,0,0,0,09:00:00.841000


In [9]:
l2_example.head()

,date,code,server_time,local_time,preclose,open,high,low,last,upper_limit,lower_limit,volume,turnover,iopv,ask_price1,ask_volume1,ask_price2,ask_volume2,ask_price3,ask_volume3,ask_price4,ask_volume4,ask_price5,ask_volume5,bid_price1,bid_volume1,bid_price2,bid_volume2,bid_price3,bid_volume3,bid_price4,bid_volume4,bid_price5,bid_volume5,time
0,20210302,v2108,2021-03-02 09:00:00.004,2021-03-02 08:59:55.529,8060,0,0,0,0,8610,7490,0,0,0,8105,1,8175,1,0,0,0,0,0,0,7880,11,7835,1,0,0,0,0,0,0,09:00:00.004000
1,20210302,v2108,2021-03-02 09:00:00.185,2021-03-02 08:59:55.733,8060,0,0,0,0,8610,7490,0,0,0,8100,2,8175,1,0,0,0,0,0,0,7880,11,7835,1,0,0,0,0,0,0,09:00:00.185000
2,20210302,v2108,2021-03-02 09:00:00.438,2021-03-02 08:59:55.978,8060,0,0,0,0,8610,7490,0,0,0,8295,1,0,0,0,0,0,0,0,0,7775,1,0,0,0,0,0,0,0,0,09:00:00.438000
3,20210302,v2108,2021-03-02 09:00:00.841,2021-03-02 08:59:56.210,8060,0,0,0,0,8610,7490,0,0,0,8095,3,8170,2,8295,1,0,0,0,0,7775,1,0,0,0,0,0,0,0,0,09:00:00.841000
4,20210302,v2108,2021-03-02 09:00:00.841,2021-03-02 08:59:56.465,8060,0,0,0,0,8610,7490,0,0,0,8095,3,8165,1,8295,1,0,0,0,0,7775,1,0,0,0,0,0,0,0,0,09:00:00.841000


In [3]:
rc.get_future_list('20210302', 'l2')

['lh2201.csv.gz',
 'c2107.csv.gz',
 'pp2107.csv.gz',
 'i2202.csv.gz',
 'bb2103.csv.gz',
 'eb2109.csv.gz',
 'm2103.csv.gz',
 'eg2202.csv.gz',
 'jd2111.csv.gz',
 'cs2111.csv.gz',
 'jd2108.csv.gz',
 'v2202.csv.gz',
 'b2109.csv.gz',
 'j2109.csv.gz',
 'l2104.csv.gz',
 'j2111.csv.gz',
 'cs2105.csv.gz',
 'i2104.csv.gz',
 'v2110.csv.gz',
 'p2112.csv.gz',
 'm2107.csv.gz',
 'IC2103.csv.gz',
 'y2109.csv.gz',
 'eg2107.csv.gz',
 'fb2201.csv.gz',
 'a2111.csv.gz',
 'pg2106.csv.gz',
 'rr2106.csv.gz',
 'y2105.csv.gz',
 'm2201.csv.gz',
 'lh2109.csv.gz',
 'fb2111.csv.gz',
 'jm2104.csv.gz',
 'fb2103.csv.gz',
 'lh2111.csv.gz',
 'eb2106.csv.gz',
 'cs2201.csv.gz',
 'eb2112.csv.gz',
 'c2103.csv.gz',
 'pp2108.csv.gz',
 'jd2110.csv.gz',
 'c2201.csv.gz',
 'i2108.csv.gz',
 'b2108.csv.gz',
 'jm2103.csv.gz',
 'pp2104.csv.gz',
 'pg2112.csv.gz',
 'bb2108.csv.gz',
 'p2201.csv.gz',
 'bb2111.csv.gz',
 'y2108.csv.gz',
 'a2107.csv.gz',
 'v2111.csv.gz',
 'j2106.csv.gz',
 'y2103.csv.gz',
 'l2105.csv.gz',
 'pg2103.csv.gz',
 